# Required Packages

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # advanced data visualization

from sklearn.preprocessing import StandardScaler#dimansity redaction
from sklearn.decomposition import PCA #PCA

from sklearn.model_selection import train_test_split # split data into train and test
from sklearn.preprocessing import MinMaxScaler # scale the data between 0 - 1
from tensorflow.keras.models import Sequential # initiate the mode 
from tensorflow.keras.layers import Dense, Activation, Dropout # add the layers
from tensorflow.keras.optimizers import Adam # optimizer 

from tensorflow.keras.callbacks import EarlyStopping # Early Stopping

from sklearn.metrics import classification_report,confusion_matrix # Model Evaluation 

# Read the Data and Check It

In [ ]:
# read the data
df = pd.read_csv('../input/breast-cancer-wisconsin-data/data.csv')


In [ ]:
# top 5 rows 
df.head()

In [ ]:
# info
df.info()

In [ ]:
# statistical distribution of various features
df.describe().round(2).transpose()

# Feature Engineering

In [ ]:
# drop Unnamed: 32
df = df.drop('Unnamed: 32', axis = 1)

In [ ]:
# Convert Diagnosis to numeric 0 - 1
df['diagnosis'] = df['diagnosis'].apply(lambda x: 1 if x == 'M' else 0)

# Exploratory Data Analysis

In [ ]:
# count plot of the diagnosis
plt.figure(figsize = (8, 4), dpi = 100)
sns.countplot(data = df, x = 'diagnosis')
plt.show()

In [ ]:
# Correlation between features 
plt.figure(figsize = (8, 4), dpi = 100)
df.corr()['diagnosis'].sort_values().plot(kind  = 'bar');

Very strong correlation, we should be able to get very good predictions based on this analysis here 

In [ ]:
# correlation analysis using heatmap
plt.figure(figsize = (8, 4), dpi = 100)
sns.heatmap(df.corr());

> # #Reduce dimensions with PCA
With the principal component analysis we only use the features, since PCA  is a unsupervised machine learning method

In [ ]:
#Standardize the data of the features
standardized = StandardScaler()
standardized

standardized.fit(df)

In [ ]:
scaled_data = standardized.transform(df)

In [ ]:
pca = PCA(n_components=3)

In [ ]:
pca.fit(scaled_data)

In [ ]:
x_pca = pca.transform(scaled_data)

In [ ]:
scaled_data.shape

In [ ]:
x_pca.shape

In [ ]:
def diag(x):
    if x =='M':
        return 1
    else:
        return 0
df_diag= df['diagnosis'].apply(diag)

> I plot the principal components  to see the relations

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
x_pca[:1]

In [ ]:
fig = plt.figure(figsize=(15, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x_pca[:,0], x_pca[:,1], x_pca[:,2], c=df_diag, s=60)
ax.legend(['Malign'])
ax.set_xlabel('First Principal Component')
ax.set_ylabel('Second Principal Component')
ax.set_zlabel('Third Principal Component')
ax.view_init(30, 120)

In [ ]:
ax = plt.figure(figsize=(12,8))
sns.scatterplot(x_pca[:,0], x_pca[:,2],hue=df['diagnosis'], palette ='Set1' )
plt.xlabel('First Principal Component')
plt.ylabel('Third Principal Component')

In [ ]:
ax = plt.figure(figsize=(12,8))
sns.scatterplot(x_pca[:,1], x_pca[:,2],hue=df['diagnosis'], palette ='Set1' )
plt.xlabel('Second Principal Component')
plt.ylabel('Third Principal Component')

In [ ]:
ax = plt.figure(figsize=(12,8))
sns.scatterplot(x_pca[:,0], x_pca[:,1],hue=df['diagnosis'], palette ='Set1' )
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')

In [ ]:
df_pc = pd.DataFrame(pca.components_, columns = df.columns)
df_pc

In [ ]:
plt.figure(figsize=(15, 8))
sns.heatmap(df_pc, cmap='viridis')
plt.title('Principal Components correlation with the features')
plt.xlabel('Features')
plt.ylabel('Principal Components')

# Model Building

In [ ]:
# seprate X and y
X = df.drop('diagnosis', axis = 1).values
y = df['diagnosis'].values 

In [ ]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=101)

In [ ]:
# intiate the scaler 
scaler = MinMaxScaler()
# fit the scaler and scale training data 
X_train= scaler.fit_transform(X_train)
# scale the test data 
X_test = scaler.transform(X_test)

# > single Dence layer

In [ ]:
model = Sequential()
model.add(Dense(30,activation='relu'))

# compile the model
model.compile(optimizer='adam',loss='binary_crossentropy');

In [ ]:
# fit the model
model.fit(x=X_train,y=y_train,validation_data=(X_test,y_test),epochs=600)

In [ ]:
# compare train and validation loss 
model_loss = pd.DataFrame(model.history.history)
plt.figure(figsize=(8,4), dpi = 100)
plt.plot(model_loss)
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

# > multi Dence layer

In [ ]:
# build model architecture
model = Sequential()
model.add(Dense(30,activation='relu'))
model.add(Dense(15,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

# compile the model
model.compile(optimizer='adam',loss='binary_crossentropy');

In [ ]:
# fit the model
model.fit(x=X_train,y=y_train,validation_data=(X_test,y_test),epochs=600)

In [ ]:
# compare train and validation loss 
model_loss = pd.DataFrame(model.history.history)
plt.figure(figsize=(8,4), dpi = 100)
plt.plot(model_loss)
plt.show()

Validation loss is increasing while training loss is decreasing. This is a strong sign of verfitting. We can use 'early stopping' or dropout to aviod overfitting  

# Dealing with Overfitting 

## Early Stopping

In [ ]:
# build model architecture
model = Sequential()
model.add(Dense(30,activation='relu'))
model.add(Dense(15,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

# compile the model
model.compile(optimizer='adam',loss='binary_crossentropy');

In [ ]:
# Define Early Stopping 
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25)

In [ ]:
# fit the model
model.fit(x=X_train,y=y_train,validation_data=(X_test,y_test),epochs=600, callbacks = [early_stop])

In [ ]:
# compare train and validation loss 
model_loss = pd.DataFrame(model.history.history)
plt.figure(figsize=(8,4), dpi = 100)
plt.plot(model_loss)
plt.show()

# add Drop Out 

In [ ]:
# build model architecture
model = Sequential()
model.add(Dense(30,activation='relu'))
model.add(Dropout(0.5)) # Drop Out Layer 
model.add(Dense(15,activation='relu'))
model.add(Dropout(0.5)) # Drop Out Layer
model.add(Dense(1,activation='sigmoid'))

# compile the model
model.compile(optimizer='adam',loss='binary_crossentropy');

In [ ]:
# fit the model
#model.fit(x=X_train,y=y_train,validation_data=(X_test,y_test),epochs=600, callbacks = [early_stop])
model.fit(x=X_train,y=y_train,validation_data=(X_test,y_test),epochs=600)

In [ ]:
# compare train and validation loss 
model_loss = pd.DataFrame(model.history.history)
plt.figure(figsize=(8,4), dpi = 100)
plt.plot(model_loss)
plt.show()

# Model Evaluation

In [ ]:
# Predictions
y_pred = model.predict(X_test)
predictions = np.round(y_pred).astype(int)

In [ ]:
# Classification Report 
print(classification_report(y_test,predictions))